In [ ]:
import requests
from bs4 import BeautifulSoup
import time
import smtplib
from email.mime.text import MIMEText
import matplotlib.pyplot as plt
from datetime import datetime
from IPython.display import display, Markdown

# Dictionary to store tracked products with price trends and alerts
tracked_products = {}

# Function to fetch product details from a URL
def fetch_product_details(url):
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36"
    }
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.content, "html.parser")
    
    try:
        name = soup.find("span", {"id": "productTitle"}).get_text(strip=True)
        price = soup.find("span", {"class": "a-price-whole"}).get_text(strip=True)
        price = float(price.replace(",", ""))
    except AttributeError:
        return None, None
    return name, price

# Function to add or update product details in the dictionary
def add_or_update_product(url):
    name, price = fetch_product_details(url)
    if name and price:
        timestamp = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        if url in tracked_products:
            old_price = tracked_products[url]['prices'][-1][1]  # Get the latest price
            if old_price != price:
                display(Markdown(f"### Price updated for {name}: ₹{old_price} → ₹{price}"))
            tracked_products[url]['prices'].append((timestamp, price))  # Add new price
        else:
            display(Markdown(f"### Adding new product: {name} at ₹{price}"))
            tracked_products[url] = {'name': name, 'prices': [(timestamp, price)], 'alert_threshold': None}
    else:
        display(Markdown("### Failed to fetch product details. Check the URL."))

# Function to set price alert threshold for a product
def set_alert_threshold(url, threshold):
    if url in tracked_products:
        tracked_products[url]['alert_threshold'] = threshold
        display(Markdown(f"### Price drop alert threshold set for {tracked_products[url]['name']} at ₹{threshold}"))
    else:
        display(Markdown("### Product URL not found."))

# Function to send email notification
def send_email_notification(product_name, old_price, new_price, product_url, sender_email, sender_password, recipient_email):
    msg = MIMEText(f"Price alert for {product_name}!\n\nOld Price: ₹{old_price}\nNew Price: ₹{new_price}\nURL: {product_url}")
    msg['Subject'] = f"Price Drop Alert for {product_name}"
    msg['From'] = sender_email
    msg['To'] = recipient_email
    
    with smtplib.SMTP('smtp.gmail.com', 587) as server:
        server.starttls()
        server.login(sender_email, sender_password)
        server.sendmail(sender_email, recipient_email, msg.as_string())

# Function to check for price drops
def check_price_drops(sender_email, sender_password, recipient_email):
    for url, product in tracked_products.items():
        name, latest_price = fetch_product_details(url)
        if name and latest_price:
            latest_price = float(latest_price) if '₹' in str(latest_price) else 0
            previous_price = float(product['prices'][-1][1].replace('₹', '').replace(',', '')) if product['prices'] else latest_price
            if product['alert_threshold'] and latest_price < product['alert_threshold'] and latest_price < previous_price:
                send_email_notification(name, previous_price, latest_price, url, sender_email, sender_password, recipient_email)
                display(Markdown(f"### Price drop detected for {name}: ₹{previous_price} → ₹{latest_price}"))

# Function to visualize price trends for a specific product
def visualize_price_trend(url):
    if url in tracked_products:
        product = tracked_products[url]
        timestamps, prices = zip(*product['prices'])  # Extract timestamps and prices
        
        # Plot the price trend
        plt.figure(figsize=(10, 6))
        plt.plot(timestamps, prices, marker='o', label=f"{product['name']} Price Trend")
        plt.xlabel('Timestamp', fontsize=12)
        plt.ylabel('Price (₹)', fontsize=12)
        plt.title(f"Price Trend for {product['name']}", fontsize=14)
        plt.xticks(rotation=45, fontsize=10)
        plt.yticks(fontsize=10)
        plt.grid(True, linestyle='--', alpha=0.6)
        plt.legend(fontsize=12)
        plt.tight_layout()
        plt.show()
    else:
        display(Markdown("### Product not found in tracked products."))

# Function to list all tracked products
def list_tracked_products():
    if tracked_products:
        display(Markdown("### Tracked Products:"))
        for url, details in tracked_products.items():
            display(Markdown(f"- **{details['name']}** - ₹{details['prices'][-1][1]} (Latest Price)\n  [Product Link]({url})"))
    else:
        display(Markdown("### No products are currently being tracked."))

# Collect user's email and password for sending alerts
sender_email = input("Enter your email address (for sending alerts): ")
sender_password = input("Enter your email password (or app password): ")
recipient_email = input("Enter recipient's email address (for receiving alerts): ")

# Interactive command-line interface for product tracking and alerts
while True:
    print("\nOptions:")
    print("1. Add/Update a Product")
    print("2. List Tracked Products")
    print("3. Visualize Price Trend")
    print("4. Set Price Alert Threshold")
    print("5. Check for Price Drops")
    print("6. Exit")
    choice = input("Enter your choice: ").strip()
    
    if choice == "1":
        product_url = input("Enter the product URL: ").strip()
        add_or_update_product(product_url)
    elif choice == "2":
        list_tracked_products()
    elif choice == "3":
        product_url = input("Enter the product URL to visualize: ").strip()
        visualize_price_trend(product_url)
    elif choice == "4":
        product_url = input("Enter the product URL to set an alert: ").strip()
        threshold = float(input("Enter the price threshold (₹): "))
        set_alert_threshold(product_url, threshold)
    elif choice == "5":
        check_price_drops(sender_email, sender_password, recipient_email)
    elif choice == "6":
        print("Exiting...")
        break
    else:
        print("Invalid choice. Please try again.")


Enter your email address (for sending alerts): ANADN
Enter your email password (or app password): @Nand21J
Enter recipient's email address (for receiving alerts): anandvishwakarma21j@gmail.com

Options:
1. Add/Update a Product
2. List Tracked Products
3. Visualize Price Trend
4. Set Price Alert Threshold
5. Check for Price Drops
6. Exit
